In [229]:
import string
import numpy as np
import pandas as pd
import seaborn as sns
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.plotting import output_notebook, figure, show
from diff_match_patch import diff_match_patch
from datasets import load_from_disk, load_dataset
from termcolor import colored

In [120]:
dataset_path = '../input/data/train_dataset/validation'

In [130]:
# ground_truth
dataset = load_from_disk(dataset_path)
gt = {example['id'] : example['answers']['text'][0] for example in dataset}

In [131]:
# predictions(EM 60%)
with open('../input/checkpoint/for_eda/predictions_valid.json', 'r') as prediction_file:
    pr = json.loads(prediction_file.read())
pr = pd.Series(pr, index=list(pr.keys()))

In [132]:
df = pd.DataFrame({'pred': pr, 'answer': gt}).reset_index()
df.columns= ['id', 'pred', 'answer']
df

,id,pred,answer
0,mrc-0-003264,한보철강,한보철강
1,mrc-0-004762,1868년,1871년
2,mrc-1-001810,나뭇잎,나뭇잎
3,mrc-1-000219,금대야,금대야
4,mrc-1-000285,수평적 관계,수평적 관계
...,...,...,...
235,mrc-0-000484,안평군,제 양왕
236,mrc-0-002095,'일곱 개의 신전 광장','일곱 개의 신전 광장'
237,mrc-0-003083,윤치호,미나미 지로
238,mrc-0-002978,"200,000명","200,000명"


---
## 예측과 정답이 다른 경우 비교해보기
- 240개중 예측과 정답이 다른 경우 -> 총 96개
    - 생각보다 형식은 비슷하다!

In [162]:
different_rows = df[df['pred']!=df['answer']]
different_rows.index = range(len(different_rows.index))
different_rows

,id,pred,answer
0,mrc-0-004762,1868년,1871년
1,mrc-0-003032,1967년 8월 16일,1967년 11월 15일
2,mrc-1-000724,1963년,1965년
3,mrc-0-003727,자칫,〈중앙일보〉
4,mrc-0-003115,이이노야 성,미타케성
...,...,...,...
91,mrc-0-004565,별도로 분리된 투표를 했다. 각 부족마다 다수결 득표,다수결
92,mrc-1-000024,물적 성과,물적 성과(物的成果)
93,mrc-0-000484,안평군,제 양왕
94,mrc-0-003083,윤치호,미나미 지로


In [235]:
dmp = diff_match_patch()

pred_in_answer = [] # 예측이 정답의 일부분인경우
answer_in_pred = [] # 예측이 정답을 포함하는 경우
is_diff_number = [] # 숫자 예측을 잘못 한 경우
is_diff_not_korean = [] # 한국어 예측을 잘못한 경우

for _, (pred, answer) in enumerate(zip(different_rows['pred'], different_rows['answer'])):
    diff = dmp.diff_main(pred, answer)
    dmp.diff_cleanupSemantic(diff)
    prediction = ''
    answer_text = ''
    mispredict = ''
    for offset in diff:
        if offset[0] == 0:
            prediction += offset[1]
            answer_text += offset[1]
        elif offset[0] == -1:
            prediction += offset[1]
            mispredict += offset[1]
        elif offset[0] == 1:
            answer_text += offset[1]
    if prediction in answer_text:
        pred_in_answer.append((prediction,answer_text))
    if answer_text in prediction:
        answer_in_pred.append((prediction,answer_text))
    if mispredict.isnumeric():
        is_diff_number.append((prediction,answer_text))
    for c in mispredict:
        if c != '' and not (ord('가') <= ord(c) <= ord('힣')) and (ord(c) != ord(' ') and c not in string.punctuation):
            is_diff_not_korean.append((prediction,answer_text))
    print(f"pred:{colored(prediction, 'blue'):<30} | answer: {colored(answer_text, 'green'):<30} | mispredict:{colored(mispredict, 'red'):<20}" )

pred:1868년                 | answer: 1871년                 | mispredict:68         
pred:1967년 8월 16일          | answer: 1967년 11월 15일         | mispredict:86         
pred:1963년                 | answer: 1965년                 | mispredict:3          
pred:자칫                    | answer: 〈중앙일보〉                | mispredict:자칫         
pred:이이노야 성                | answer: 미타케성                  | mispredict:이이노야       
pred:4순위                   | answer: 전체 4순위                | mispredict:           
pred:유형 준융합성 천연두           | answer: 보통 유형 준융합성 천연두        | mispredict:           
pred:기와조각                  | answer: '진전(陳田)'이라 새겨진 기와조각   | mispredict:           
pred:브라질 포르투갈어             | answer: 유럽 포르투갈어              | mispredict:브라질        
pred:릿지당                   | answer: 메이저 릿지                | mispredict:당          
pred:요희(淖姬)                | answer: 요희                    | mispredict:(淖姬)       
pred:항일운동참여 경력             | answer: 만주국 관리                | mispredict:항일운동

**타입을 나눠보자면 다음과 같이 나눠볼 수 있을 것 같습니다.**
### 1. 유사관계 : 형식은 맞지만 디테일이 다른 경우
    - pred: 1967년 8월 16일 answer: 1967년 11월 15일 mispredict: 86 => 날짜형식은 맞추었지만 정확한 숫자가 틀림
    - pred: 이이노야 성 answer: 미타케성 mispredict: 이이노야 => 성이라는 형식은 맞추었지만 정확한 이름이 틀림
    - pred:이치키 마츠히로  answer: 신도 케이  mispredict:이치키 마츠히로
       
### 2. 포함관계 : 정답 어구의 일부분만을 예측했거나, 정답 어구를 포함한 어구를 예측한 경우
    1. 정답어구의 일부분만을 예측한 경우
        - pred: 4순위 answer: 전체 4순위 mispredict
        - _pred: 베게티우스 answer: 작가 베게티우스 mispredict:_
            - 이 경우에는 과연 틀렸다고 봐야하는 걸까요? 의문점이 남습니다.
        - pred: 망치 answer: 쇠망치 mispredict:
        - pred: 걸프 카르텔 answer: 걸프 카르텔(카르텔 델 골포: Cartel del golfo) mispredict:
        - pred: 도로스 answer: 대형 항공 모함 도로스 mispredict:
    2. 정답 어구를 포함하여 너무 많은 부분을 예측으로 내놓은 경우 --> 이 경우는 에폭을 늘려 학습을 더 했어야 했던 것일까요?
        - pred: 나말여초 불상 answer: 나말여초 mispredict:  불상
        - pred: 1945년과 1960년대 후반 answer: 1945년 mispredict: 과 1960년대 후반
        - _pred: 젊은 근위대 answer: ＜젊은 근위대＞ mispredict:_ => 아마도 이러한 경우는 LB상에서는 정답처리 될 듯합니다.
        - pred: 탄광 환경 문제 answer: 탄광 mispredict:  환경 문제
        - pred: 별도로 분리된 투표를 했다. 각 부족마다 다수결 득표 answer: 다수결 mispredict: 별도로 분리된 투표를 했다. 각 부족마다 다수결 득표

### 3. _언어는 이해했으나 맥락을 파악못한 경우(주목할만한 경우)_
    - **동일한 글자매칭이 없는 경우에도 의미상 비슷한 종류의 단어를 캐치한 사례가 있었습니다.(언어 학습이 어느정도 잘 되었다는 것을 의미)**
    - pred: 수원 answer: 울산 mispredict: 수원
    - pred: 우익진영 answer: 공산당 mispredict: 우익진영
    - pred: 범퍼 answer: 트렁크 mispredict: 범퍼
    - pred:안평군 answer: 제 양왕  mispredict:안평군
    - pred:대부업 answer: '사채회사' mispredict:대부업

### 4. 아예 이해를 하지 못한 경우 -> 정답의 근거가 되는 context를 살펴보고 추가적인 EDA가 더 필요합니다.
    - pred:비록  answer: 가루아  mispredict:비록
    - pred:때론  answer: 사춘기에서 흔히 볼 수 있는 정서적 불안정성 mispredict:때론
    - pred:오로지  answer: 모든 관직을 거친 대과 급제자 mispredict:오로지
    
### 5. 서술형
    - pred:“어떤 경우에도 행할 수” answer: 본국 의회가 식민지에 대한 입법을 “어떤 경우에도 행할 수” 있도록 하였다  => 대부분 단답형의 example로 학습한 BERT가 맞추기 어려운 서술형의 답
    - pred:상급자 수준의  answer: 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문이다 mispredict:상급자 수준의

In [211]:
print("포함관계 - 정답의 일부분만을 예측한 경우")
print(f"총 {len(pred_in_answer)}개")
for (pred, answer) in pred_in_answer:
    print(colored(pred,'red'), colored(answer, 'green'))

포함관계 - 정답의 일부분만을 예측한 경우
총 22개
4순위 전체 4순위
유형 준융합성 천연두 보통 유형 준융합성 천연두
기와조각 '진전(陳田)'이라 새겨진 기와조각
베게티우스 작가 베게티우스
망치 쇠망치
여정현 여정현(呂正鉉)
펜실베이니아주 펜실베이니아주 출신
전극 높은 전위의 전극
브리튼인 브리튼인들
도로스 대형 항공 모함 도로스
교수형 성벽에 목이 매달리는 교수형
“어떤 경우에도 행할 수” 본국 의회가 식민지에 대한 입법을 “어떤 경우에도 행할 수” 있도록 하였다
태화관 태화관(서울시 종로구 인사동 소재)
걸프 카르텔 걸프 카르텔(카르텔 델 골포: Cartel del golfo)
골룸바노 아일랜드 선교자 골룸바노
마거릿 대처 마거릿 대처 전 영국수상
문치미르 트르피미로비치 왕조(Trpimirović) 출신의 문치미르
'달빛 정원' '달빛 정원'(Moonlight Garden)
젊은 근위대 ＜젊은 근위대＞
광휘에의 각성 "광휘에의 각성"(코키에노 메자메)
앨리스 페어살 스미스 퀘이커 교도였던 앨리스 페어살 스미스
물적 성과 물적 성과(物的成果)


In [213]:
print("포함관계 - 정답을 포함하여 너무 많은 부분을 예측한 경우")
print(f"총 {len(answer_in_pred)}개")
for (pred, answer) in answer_in_pred:
    print(colored(pred,'red'), colored(answer, 'green'))

포함관계 - 정답을 포함하여 너무 많은 부분을 예측한 경우
총 26개
요희(淖姬) 요희
나말여초 불상 나말여초
퀘이커 교도 퀘이커 교
지분 10% 지분
스탈린그라드 전투에서 독일군의 대참패 독일
제 3자의 제 3자
가리타 히사노리에 가리타 히사노리
무위태수 관구흥(毌丘興) 무위태수
크라운라이터의 사카이 야스유키 크라운라이터
경애왕과 경순왕 경순왕
바르톨로메오 브뤼기에르 신부 바르톨로메오 브뤼기에르
34자의 34자
혼묘지(本妙寺) 혼묘지
스텐코프의 혹 혹
특정한 귀인 평향과 행동의 상관 관계 귀인 평향과 행동의 상관 관계
1945년과 1960년대 후반 1945년
제비초리형태 제비초리
보은군 남쪽 상용리(현 영동군 용산면 상용리) 마을 근처 보은군 남쪽 상용리(현 영동군 용산면 상용리) 마을
현상금 사냥꾼들의 식재료 식재료
탄광 환경 문제 탄광
디지털 컴퓨터가 등장 디지털 컴퓨터
란다우 튜브들이 란다우 튜브
카바레나 클럽 등 카바레나 클럽
보수적 보수
리퍼와 솔저: 76 리퍼
별도로 분리된 투표를 했다. 각 부족마다 다수결 득표 다수결


In [236]:
# 한자가 붙은 경우는 LB 스코어에서 정답처리 될까요?
is_diff_not_korean

[('1868년', '1871년'),
 ('1868년', '1871년'),
 ('1967년 8월 16일', '1967년 11월 15일'),
 ('1967년 8월 16일', '1967년 11월 15일'),
 ('1963년', '1965년'),
 ('요희(淖姬)', '요희'),
 ('요희(淖姬)', '요희'),
 ('사이넨지(西念寺)', '도고쿠'),
 ('사이넨지(西念寺)', '도고쿠'),
 ('사이넨지(西念寺)', '도고쿠'),
 ('1922년', '1932년'),
 ('지분 10%', '지분'),
 ('지분 10%', '지분'),
 ('무위태수 관구흥(毌丘興)', '무위태수'),
 ('무위태수 관구흥(毌丘興)', '무위태수'),
 ('무위태수 관구흥(毌丘興)', '무위태수'),
 ('혼묘지(本妙寺)', '혼묘지'),
 ('혼묘지(本妙寺)', '혼묘지'),
 ('혼묘지(本妙寺)', '혼묘지'),
 ('1945년과 1960년대 후반', '1945년'),
 ('1945년과 1960년대 후반', '1945년'),
 ('1945년과 1960년대 후반', '1945년'),
 ('1945년과 1960년대 후반', '1945년'),
 ('8기', '1기'),
 ('4년', '60년간'),
 ('초성(初成)시기', '1895년'),
 ('초성(初成)시기', '1895년'),
 ('리퍼와 솔저: 76', '리퍼'),
 ('리퍼와 솔저: 76', '리퍼')]